In [1]:
import numpy as np
import pandas as pd
import os
import re
import unidecode

from gensim.test.utils import get_tmpfile
from gensim.models import Word2Vec

In [2]:
data_dir = "Data"

In [3]:
df = pd.read_csv(os.path.join(data_dir,"train.csv"))

In [4]:
#Primera prueba con el 20% del Dataframe
df_short = df.sample(frac=1)
df_short.shape

(1804874, 45)

In [5]:
texts = df_short["comment_text"].values 

## Preprocessing

In [6]:
from nltk.tokenize import word_tokenize, TreebankWordTokenizer, WordPunctTokenizer

def tokenize_text(text, word=True, tokenizer=None):
    if tokenizer is None and word:
        tokens = word_tokenize(text)
    elif tokenizer is None and not word:
        tokens = sent_tokenize(text)
    else:
        tokens = tokenizer.tokenize(text)
    return tokens 

In [7]:
def convert_characters(text, style='lower'):
    if style == 'lower':
        out = text.lower()
    else:
        out = text.upper()
    return out

In [8]:
def remove_blanks(tokens):
    return [token.strip() for token in tokens]

In [9]:
def remove_accents(tokens):
    tokens = [unidecode.unidecode(token) for token in tokens]
    return(tokens)

In [10]:
english_contractions_mapping = {
    "ain't": "is not", "aren't": "are not","can't": "cannot", 
    "can't've": "cannot have", "'cause": "because", "could've": "could have", 
    "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
    "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
    "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
    "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
    "he'll've": "he he will have", "he's": "he is", "how'd": "how did", 
    "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
    "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
    "I'll've": "I will have","I'm": "I am", "I've": "I have", 
    "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
    "i'll've": "i will have","i'm": "i am", "i've": "i have", 
    "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
    "it'll": "it will", "it'll've": "it will have","it's": "it is", 
    "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
    "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
    "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
    "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
    "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
    "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
    "she's": "she is", "should've": "should have", "shouldn't": "should not", 
    "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
    "this's": "this is",
    "that'd": "that would", "that'd've": "that would have","that's": "that is", 
    "there'd": "there would", "there'd've": "there would have","there's": "there is", 
    "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
    "they'll've": "they will have", "they're": "they are", "they've": "they have", 
    "to've": "to have", "wasn't": "was not", "we'd": "we would", 
    "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
    "we're": "we are", "we've": "we have", "weren't": "were not", 
    "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
    "what's": "what is", "what've": "what have", "when's": "when is", 
    "when've": "when have", "where'd": "where did", "where's": "where is", 
    "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
    "who's": "who is", "who've": "who have", "why's": "why is", 
    "why've": "why have", "will've": "will have", "won't": "will not", 
    "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
    "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
    "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
    "you'll've": "you will have", "you're": "you are", "you've": "you have"
} 

In [11]:
def expand_match(contraction): 
        match = contraction.group(0) 
        first_char = match[0] 
        expanded_contraction = english_contractions_mapping.get(match) if english_contractions_mapping.get(match) else english_contractions_mapping.get(match.lower())                        
        expanded_contraction = first_char+expanded_contraction[1:] 
        return expanded_contraction 
    
def expand_contractions(sentence, english_contractions_mapping):    
    contractions_pattern = re.compile('({})'.format('|'.join(english_contractions_mapping.keys())),
                                      flags=re.IGNORECASE|re.DOTALL) 
    return contractions_pattern.sub(expand_match, sentence)    

In [12]:
from nltk.corpus import stopwords

In [13]:
english_nltk_stopwords = stopwords.words('english')

In [14]:
def remove_stopwords(tokens, stopwords_list):
    return [token for token in tokens if token not in stopwords_list]

In [15]:
import string
def remove_punct(tokens):
    out = [''.join(c for c in s if c not in string.punctuation) for s in tokens]
    out = [s for s in out if s]
    return out

In [16]:
from nltk.stem import WordNetLemmatizer
def lemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    out=[]
    for token in tokens:
        out.append(lemmatizer.lemmatize(token))
    return out

### Applying preprocessing functions

In [17]:
texts_2 = []
for text in texts:
    texts_2.append(expand_contractions(text,english_contractions_mapping))

In [18]:
print(texts[2])

.
"But now, the man seen by many as too left-wing, too boring and too out of touch "

But now, it has been delivered to the Brits in no uncertain terms that it is in fact Ms. May who is boring, out of touch while  the Conservatives that are taking a political beating for their self inflicted wound, aka; Brexit.

Make no mistake, a good majority of the Con MP's in the House were in support of Brexit. 

They just don't wants to talk about it anymore. I wonder WHY????

.


In [19]:
texts_2[2]

'.\n"But now, the man seen by many as too left-wing, too boring and too out of touch "\n\nBut now, it has been delivered to the Brits in no uncertain terms that it is in fact Ms. May who is boring, out of touch while  the Conservatives that are taking a political beating for their self inflicted wound, aka; Brexit.\n\nMake no mistake, a good majority of the Con MP\'s in the House were in support of Brexit. \n\nThey just do not wants to talk about it anymore. I wonder WHY????\n\n.'

In [20]:
texts_3=[]
for text in texts_2:
    texts_3.append(convert_characters(text))

In [21]:
texts_3[2]

'.\n"but now, the man seen by many as too left-wing, too boring and too out of touch "\n\nbut now, it has been delivered to the brits in no uncertain terms that it is in fact ms. may who is boring, out of touch while  the conservatives that are taking a political beating for their self inflicted wound, aka; brexit.\n\nmake no mistake, a good majority of the con mp\'s in the house were in support of brexit. \n\nthey just do not wants to talk about it anymore. i wonder why????\n\n.'

In [22]:
tokens=[]
for text in texts_3:
    tokens.append(tokenize_text(text))

In [23]:
tokens[2]

['.',
 '``',
 'but',
 'now',
 ',',
 'the',
 'man',
 'seen',
 'by',
 'many',
 'as',
 'too',
 'left-wing',
 ',',
 'too',
 'boring',
 'and',
 'too',
 'out',
 'of',
 'touch',
 '``',
 'but',
 'now',
 ',',
 'it',
 'has',
 'been',
 'delivered',
 'to',
 'the',
 'brits',
 'in',
 'no',
 'uncertain',
 'terms',
 'that',
 'it',
 'is',
 'in',
 'fact',
 'ms.',
 'may',
 'who',
 'is',
 'boring',
 ',',
 'out',
 'of',
 'touch',
 'while',
 'the',
 'conservatives',
 'that',
 'are',
 'taking',
 'a',
 'political',
 'beating',
 'for',
 'their',
 'self',
 'inflicted',
 'wound',
 ',',
 'aka',
 ';',
 'brexit',
 '.',
 'make',
 'no',
 'mistake',
 ',',
 'a',
 'good',
 'majority',
 'of',
 'the',
 'con',
 'mp',
 "'s",
 'in',
 'the',
 'house',
 'were',
 'in',
 'support',
 'of',
 'brexit',
 '.',
 'they',
 'just',
 'do',
 'not',
 'wants',
 'to',
 'talk',
 'about',
 'it',
 'anymore',
 '.',
 'i',
 'wonder',
 'why',
 '?',
 '?',
 '?',
 '?',
 '.']

In [24]:
clean_toks=[]
for row in tokens:
    clean_toks.append(remove_stopwords(row, english_nltk_stopwords))

In [25]:
clean_toks[2]

['.',
 '``',
 ',',
 'man',
 'seen',
 'many',
 'left-wing',
 ',',
 'boring',
 'touch',
 '``',
 ',',
 'delivered',
 'brits',
 'uncertain',
 'terms',
 'fact',
 'ms.',
 'may',
 'boring',
 ',',
 'touch',
 'conservatives',
 'taking',
 'political',
 'beating',
 'self',
 'inflicted',
 'wound',
 ',',
 'aka',
 ';',
 'brexit',
 '.',
 'make',
 'mistake',
 ',',
 'good',
 'majority',
 'con',
 'mp',
 "'s",
 'house',
 'support',
 'brexit',
 '.',
 'wants',
 'talk',
 'anymore',
 '.',
 'wonder',
 '?',
 '?',
 '?',
 '?',
 '.']

In [26]:
clean_toks2=[]
for row in clean_toks:
    clean_toks2.append(remove_punct(row))

In [27]:
clean_toks2[2]

['man',
 'seen',
 'many',
 'leftwing',
 'boring',
 'touch',
 'delivered',
 'brits',
 'uncertain',
 'terms',
 'fact',
 'ms',
 'may',
 'boring',
 'touch',
 'conservatives',
 'taking',
 'political',
 'beating',
 'self',
 'inflicted',
 'wound',
 'aka',
 'brexit',
 'make',
 'mistake',
 'good',
 'majority',
 'con',
 'mp',
 's',
 'house',
 'support',
 'brexit',
 'wants',
 'talk',
 'anymore',
 'wonder']

In [28]:
clean_toks3=[]
for row in clean_toks2:
    clean_toks3.append(lemmatize(row))

In [29]:
clean_toks3[2]

['man',
 'seen',
 'many',
 'leftwing',
 'boring',
 'touch',
 'delivered',
 'brit',
 'uncertain',
 'term',
 'fact',
 'm',
 'may',
 'boring',
 'touch',
 'conservative',
 'taking',
 'political',
 'beating',
 'self',
 'inflicted',
 'wound',
 'aka',
 'brexit',
 'make',
 'mistake',
 'good',
 'majority',
 'con',
 'mp',
 's',
 'house',
 'support',
 'brexit',
 'want',
 'talk',
 'anymore',
 'wonder']

## Train Word2Vec

In [30]:
path = get_tmpfile("word2vec.model")

model = Word2Vec(clean_toks3, size=2, window=5, min_count=1, workers=4)
model.save("word2vec100.model")

In [31]:
trump_vector = model.wv['trump']
print(trump_vector)

[-3.9908543 -4.3346076]


In [32]:
embeds={}
for word in model.wv.vocab.keys():
    embeds[word]=model.wv[word]

In [33]:
print(len(embeds),"words")

477143 words
